In [18]:
import random
import numpy as np
import pandas as pd
import joblib
from functools import partial
from sentence_transformers import SentenceTransformer, util
from sklearn.neighbors import NearestNeighbors, KDTree
from utils import process_doc, show_example
from datasets import load_dataset

MODEL_NAME = 'paraphrase-xlm-r-multilingual-v1'
pd.options.display.max_colwidth = 500

N_SAMPLES = 11_000
MIN_DOC_LEN = 10
np.random.seed(0)
random.seed(0)

In [10]:
model = SentenceTransformer(MODEL_NAME)

In [11]:
reviews = load_dataset('amazon_polarity')['test']['content']
reviews = random.sample(
    list(filter(lambda x: len(x.split(' ')) > MIN_DOC_LEN, reviews)),
    k=N_SAMPLES
)

Reusing dataset amazon_polarity (/home/honza/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/ac31acedf6cda6bc2aa50d448f48bbad69a3dd8efc607d2ff1a9e65c2476b4c1)


In [12]:
def transform_doc(doc, model):
    processed_doc = process_doc(doc, tokenize=False)
    return model.encode(processed_doc)

In [13]:
review_embeddings = np.vstack(
    [
        transform_doc(doc, model)
        for doc in reviews
    ]
)

In [19]:
with open('embeddings.joblib', 'wb') as f:
    joblib.dump(review_embeddings, f)

In [24]:
with open('embeddings.joblib', 'rb') as f:
    np.testing.assert_array_equal(review_embeddings, joblib.load(f))

In [14]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(review_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [16]:
show_example(
    i=0,
    docs=reviews,
    p_transform_doc=partial(
        transform_doc,
        model=model,
    ),
    knn=knn,
)

text,cosine
"I own a small residential cleaning company and bought two of these. They worked great on carpet, and they look pretty cool, too. Easy to open to get rid of the dirt and debris and not too loud. But both broke within 2 months. Both of the hoses pulled away (fixed with a hose clamp) and the way the debris container fits into the body of the vacuum - you have to tilt it very slightly then pull it out - it was a given that someone was going to pull it straight out and break off the little plastic piece eventually - which they did. Going for something a little more substantial next time, although I think these might be good for someone with mostly carpeting, who lived in a ranch-style house with only one floor (a bit heavy to carry up stairs).",0.352959
"This is an excellent vacuum for those with only a few rooms of carpeting, especially if those rooms are spread across more than one floor of the house. It's nice and portable and easy to carry from room to room and up and down stairs. Also works very well. Perfect for a smaller residence or a house with only a few carpeted rooms.I am very disappointed in the quality, however. The handle is already shot after only a couple of years of very light use, such that it doesn't fold down anymore. The telescoping portion also sometimes doesn't lock. The cord winder doesn't work very well either. Not the quality I expected from Electrolux especially at this price.The bags are small, as others have noted, so don't expect to be using it a lot if you don't want to pay through the nose replacing the bags.",0.368726
"This vacuum was recommended by Consumer Reports. It is the worst product I have ever owned. I have new carpet and own 3 dogs. The internal pipes of the vacuum would clog after 3 swipes over my carpet. The filter system over the canister is where the problem starts. The canister is also very difficult to empty. It is designed so the user must reach INTO the dirt that is collected in order to turn a nob to free most of the remaining dirt!! Dust goes everywhere: in the air, in your lungs, on your clothes, in the air and back into the house!If you need a heavy duty vacuum, theBissell 5770 Healthy Home Bagless Upright Vacuumis a much better buy in every way.",0.401649
"I purchased this vacuum based on the (mostly) good remarks here on Amazon and also due to the high rating by Consumer Reports.I'll start off with the positives. I thought it was very easy to push even with the carpet setting set to low pile. It is very quiet compared to my older Hoover vacuum. The cord is nice and long too.Now the negatives. Well, there is really only one negative. I have two short hair cats who shed a normal amount. When vacuuming, most of the hair FAILS TO GO INTO THE BAG and gets blown into the compartment which holds the bag. So when you go to change the bag, there is a mess of hair and dirt to greet you. Total junk. I called my local vacuum repair shop to see what can be done and he advised me to return it ASAP since he's been flooded with the Windtunnel 2 complaints. This ain't your mother's Hoover. It's shameful how a good company turned horrible.",0.414536
"I have owned this vacuum for about 3 or 4 years. When I first got it, I was extremely happy. That only lasted for a short while. This vacuum clogs easy and takes tons of time to take care of. There is a filter that you have to replace that is a pain to find in stores, you constantly have to clean the other filter by hand (gross), and it's heavier than most vacuums that I have owned. The duster on mine stopped working about a year after purchase. What are the upsides? Well, that will be when I can afford to replace this thing.",0.417436
"This is a great little vacuum! I needed something easy to get out everyday to pick up after 2 young kids, a cat & a dog- it picks up just about everything I need it too, including the pet hair & cheerios! It is very versatile, I use on hard wood, carpet in bedrooms, area rugs & even tile floors! The h